**Домашнее задание к вебинару 5. Part-of-Speech Tagger, NER**

Задание 1. Написать теггер на данных с русским языком проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов сравнить все реализованные методы сделать выводы


In [1]:
! pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

In [3]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-06-23 12:39:49--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   194MB/s    in 0.2s    

2022-06-23 12:39:50 (194 MB/s) - ‘ru_syntagrus-ud-train-a.conllu’ saved [40736565/40736565]

--2022-06-23 12:39:50--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [4]:
# Загружаем данные
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [5]:
# Готовим тренировочную выборку:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
# Тестовую выборку:
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

# И выборку, где будут только токены   
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [6]:
fdata_train[:1]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')]]

In [7]:
# Обучим Unigram tagger

unigram_tagger = UnigramTagger(fdata_train)
accuracy_unigram = unigram_tagger.evaluate(fdata_test)
display(accuracy_unigram)

0.823732013802982

In [8]:
# Обучим Bigram tagger

bigram_tagger = BigramTagger(fdata_train)
accuracy_bigram = bigram_tagger.evaluate(fdata_test)
display(accuracy_bigram)

0.6093886320724006

In [9]:
# Обучим Trigram tagger

trigram_tagger = TrigramTagger(fdata_train)
accuracy_trigram = trigram_tagger.evaluate(fdata_test)
display(accuracy_trigram)

0.1778631421316492

In [10]:
# Обучим комбинацию тэггеров

tagger_combinations = [[UnigramTagger, BigramTagger], 
                       [UnigramTagger, TrigramTagger], 
                       [BigramTagger, TrigramTagger], 
                       [UnigramTagger, BigramTagger, TrigramTagger]]

def backoff_tagger(train_sents, taggers_list, backoff=None):
    for cls in taggers_list:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

accuracy_combs = []
backoff = DefaultTagger('NN') 
for comb in tagger_combinations:
  tag = backoff_tagger(fdata_train,
                       comb,
                       backoff = backoff) 
  accuracy_combs.append(tag.evaluate(fdata_test))

accuracy_combs

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [11]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 
                         'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 
                         'UnigramTagger+BigramTagger+TrigramTagger'], 
              'Accuracy' : [accuracy_unigram, accuracy_bigram, accuracy_trigram] + accuracy_combs}).sort_values(by = 'Accuracy', ascending=False)

result

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


Наилучший результат показала комбинация тэггеров UnigramTagger+BigramTagger+TrigramTagger.

Далее, напишем свой тэггер

In [12]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [13]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [14]:
#Строим модель с разными вариантами векторайзеров

vectorizers_char = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 

vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 

n_features = [500, 1000, 2000]

vectorizers_hash_char = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers_char + vectorizers_word + vectorizers_hash_char + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:110000])
    
    lr = LogisticRegression(random_state=42, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:110000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:110000], pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:110000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     10688
         ADP       0.98      1.00      0.99      9819
         ADV       0.92      0.90      0.91      5695
         AUX       0.81      0.97      0.88      1007
       CCONJ       0.88      0.98      0.93      4074
         DET       0.88      0.75      0.81      2841
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     26115
      NO_TAG       1.00      1.00      1.00       195
         NUM       0.86      0.90      0.88      1378
        PART       0.95      0.78      0.86      3600
        PRON       0.83      0.89      0.86      5113
       PROPN       0.78      0.58      0.66      4107
       PUNCT       1.00      1.00      1.00     21011
       SCONJ       0.80      0.91      0.85      2081
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [15]:
# Сравним результаты разных моделей

result_vectorizers = pd.DataFrame({'Vectorizer': vectorizers_char + vectorizers_word + vectorizers_hash_char + vectorizers_hash_word,
                            'F1_score': f1_scores,
                            'Accuracy': accuracy_scores})
display(result_vectorizers)

,Vectorizer,F1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.926843,0.928582
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.920009,0.922527
2,"HashingVectorizer(analyzer='char', n_features=...",0.881258,0.884200
3,"CountVectorizer(ngram_range=(1, 3))",0.646719,0.638336
4,"TfidfVectorizer(ngram_range=(1, 3))",0.663256,0.653709
5,"HashingVectorizer(n_features=1000, ngram_range...",0.343273,0.363418
6,"HashingVectorizer(analyzer='char', n_features=...",0.844096,0.849564
7,"HashingVectorizer(analyzer='char', n_features=...",0.881258,0.884200
8,"HashingVectorizer(analyzer='char', n_features=...",0.894470,0.898155
9,"HashingVectorizer(n_features=500, ngram_range=...",0.303205,0.336064


In [16]:
result_vectorizers.sort_values('F1_score', ascending=False).head(1)

,Vectorizer,F1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.926843,0.928582


In [17]:
result_vectorizers.sort_values('Accuracy', ascending=False).head(1)

,Vectorizer,F1_score,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.926843,0.928582


Наилучший результат среди всех рассмотренных подходов показал CountVectorizer

**Задание 2. Проверить насколько хорошо работает NER.**

данные брать из http://www.labinform.ru/pub/named_entities/

- проверить NER из nltk и spacy
- написать свой нер попробовать разные подходы
- передаём в сетку токен и его соседей
- передаём в сетку только токен свой вариант сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

In [21]:
! pip install corus
! pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import os
import corus
from glob import glob
import random
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
from razdel import tokenize 

import matplotlib
%matplotlib inline

import spacy
from spacy import displacy

import csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
# Загружаем данные
!wget http://www.labinform.ru/pub/named_entities/collection3.zip
!unzip collection3.zip

# Оставляем только текстовые файлы
text_files = sorted([file for file in os.listdir('/content/Collection3/') if file.endswith(".txt")])
text_files[:3]

--2022-06-23 12:58:25--  http://www.labinform.ru/pub/named_entities/collection3.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1887130 (1.8M) [application/zip]
Saving to: ‘collection3.zip’

collection3.zip     100%[===================>]   1.80M  2.09MB/s    in 0.9s    

2022-06-23 12:58:27 (2.09 MB/s) - ‘collection3.zip’ saved [1887130/1887130]

Archive:  collection3.zip
   creating: Collection3/
  inflating: Collection3/001.ann     
  inflating: Collection3/001.txt     
  inflating: Collection3/002.ann     
  inflating: Collection3/002.txt     
  inflating: Collection3/003.ann     
  inflating: Collection3/003.txt     
  inflating: Collection3/004.ann     
  inflating: Collection3/004.txt     
  inflating: Collection3/005.ann     
  inflating: Collection3/005.txt     
  inflating: Collection3/006.ann     
  inflating: Collection

['001.txt', '002.txt', '003.txt']

In [24]:
# Формируем датасет
texts = []
for text_file in text_files:
    text = open('/content/Collection3/' + text_file, encoding='utf-8')
    text = text.read()
    texts.append(text)
    
data_text = pd.DataFrame({'text': texts })
data_text

,text
0,Россия рассчитывает на конструктивное воздейст...
1,Комиссар СЕ критикует ограничительную политику...
2,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,4 октября назначены очередные выборы Верховног...
4,Следственное управление при прокуратуре требуе...
...,...
995,"Депутат от ""ЕР"": К отставке А.Сердюкова причас..."
996,\nСи Цзиньпин избран генсеком Коммунистической...
997,"""Ведомости"" узнали о смене лидера московских е..."
998,СМИ узнали о кутежах туркменского чиновника на...


Реализуем **nltk ner**

In [28]:
article = data_text['text'][0]
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(article))) if hasattr(chunk, 'label')}

{('МИД России', 'ORGANIZATION'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('России Григорий Карасин', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Тбилиси', 'PERSON')}

In [29]:
# Сравним с корректной разметкой
pd.read_csv('/content/Collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


Модель распознала не все сущности,
далее реализуем **spacy ner**

In [39]:
! python -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.1 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 8.2 MB 10.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [40]:
nlp = spacy.load("ru_core_news_sm") 
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

Spacy отработал гораздо лучше, чем nltk ner